# Get the Stock List

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def get_stock_list():
    
    # We use MalaysiaStock.biz to scrap the stock lists in Malaysia 
    url = "https://www.malaysiastock.biz/Stock-Screener.aspx"
    response = requests.get(url, headers={'User-Agent':'test'})
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find(id = "MainContent2_tbAllStock") # table id
    
    # Return the result in a list 
    stock_list = table.find_all ("a")
    return [stock_code.get ("href") [-4:] for stock_code in stock_list]

# Create Technical Indicators

In [ ]:
import yfinance as yf

In [ ]:
# Returns stock price of a ticker
def get_stock_price (ticker):
    data = yf.download (ticker)
    return data 

In [ ]:
# Returns the EMA
def add_EMA (price, day):
    return price.ewm (span = day).mean()

In [ ]:
# Calculate stochastic oscillator 
def add_STOCH (close, low, high, period, k, d = 0):
    STOCH_K = (
                (close - low.rolling (window = period).min ()) 
               / (high.rolling (window = period).max () - low.rolling (window = period).min ())
               ) * 100
    
    STOCH_K = STOCH_K.rolling (window = k).mean()
    if d == 0:
        return STOCH_K
    else:
        STOCH_D = STOCH_K.rolling (window = d).mean()
        return STOCH_D 

# Trading Strategy

In [ ]:
# function to check for EMA Bounce
def check_bounce_EMA(df):
  candle1 = df.iloc[-1]
  candle2 = df.iloc[-2]
  
  # Check if EMA18 > EMA50 > EMA100
  cond1 = candle1['EMA18'] > candle1['EMA50'] > candle1['EMA100']
  
  # Check if both stochastic oscillator are below 30
  cond2 = candle1['STOCH_%K(5,3,3)'] <= 30 or candle1['STOCH_%D(5,3,3)'] <= 30
  
  # Check for bounce
  cond3 = candle2['Low'] < candle2['EMA50'] and \
          candle2['Close'] > candle2['EMA50'] and \
          candle1['Low'] > candle1 ['EMA50']
          
  return cond1 and cond2 and cond3

# Screening the Stocks

In [ ]:
# A list to store the screened results
screened_list = []

# Get the full stock list
stock_list = get_stock_list()

In [ ]:
for stock_code in stock_list:
    try:
        # Step 1: Get stock price for each stock
        price_chart_df = get_stock_price(stock_code + ".KL")


        # Step 2: Load the indicators to the dataframe
        open = price_chart_df ["Open"]
        close = price_chart_df ["Close"]
        high = price_chart_df ["High"]
        low = price_chart_df ["Low"]

        price_chart_df ["EMA18"] = add_EMA (close, 18)
        price_chart_df ["EMA50"] = add_EMA (close, 50)
        price_chart_df ["EMA100"] = add_EMA (close, 100)
        price_chart_df ["STOCH_%K(5,3,3)"] = add_STOCH (close, low, high, 5,3)
        price_chart_df ["STOCH_%D(5,3,3)"] = add_STOCH (close, low, high, 5,3,3)
        
        # If all 3 conditions are met, add stock into the screened list
        if check_bounce_EMA (price_chart_df):
            screened_list.append (stock_code)
            print (screened_list)
    
    # Print error if any
    except Exception as e:
        print (e)

In [ ]:
screened_list

# Email the Screening Results

In [ ]:
import email
import smtplib

In [ ]:
# Set up email fields
msg = email.message_from_string(", ".join(screened_list))

# Input the email address
email_add = "EMAIL ADDRESS"

msg['From'] = email_add
msg['To'] = email_add
msg['Subject'] = "EMA Bounce Result for Today!"

In [ ]:
# Set up SMTP Connection 
'''
for gmail user: s = smtplib.SMTP("smtp.gmail.com",587)
for yahoo mail user: s = smtplib.SMTP("smtp.mail.yahoo.com",587) 
for hotmail user: s = smtplib.SMTP("smtp.live.com",587)
'''
s = smtplib.SMTP("smtp.live.com",587)
s.ehlo() 
s.starttls()
s.ehlo()

email_from = email_add
email_to = email_add

# Enter email password
s.login(email_from,"PASSWORD")

s.sendmail(email_from, [email_to], msg.as_string())
s.quit()